In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
advanced_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

agent = create_agent(
    model=basic_model,  # Default model
    # tools=tools,
    middleware=[dynamic_model_selection]
)

In [3]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "1 + 1"}]}
)

{'messages': [HumanMessage(content='1 + 1', additional_kwargs={}, response_metadata={}, id='48ac4a66-06a2-4ac5-a5c4-daf10bbcab7e'),
  AIMessage(content='1 + 1 = 2', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--511acb45-6d8f-4956-a029-7fe2bd2134c3-0', usage_metadata={'input_tokens': 5, 'output_tokens': 7, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})]}

In [5]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "25에 10을 더하고, 그 결과에 3을 곱한 값은 얼마입니까?"}]}
)

{'messages': [HumanMessage(content='25에 10을 더하고, 그 결과에 3을 곱한 값은 얼마입니까?', additional_kwargs={}, response_metadata={}, id='2155ce8f-0fb7-482f-9e73-2ead7711e492'),
  AIMessage(content='이 문제는 두 단계로 계산해야 합니다.\n\n1.  **25에 10을 더합니다:**\n    25 + 10 = 35\n\n2.  **그 결과에 3을 곱합니다:**\n    35 * 3 = 105\n\n따라서 25에 10을 더하고, 그 결과에 3을 곱한 값은 **105**입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--e2242d46-5554-4962-aff8-ad02509193a3-0', usage_metadata={'input_tokens': 25, 'output_tokens': 93, 'total_tokens': 118, 'input_token_details': {'cache_read': 0}})]}

In [6]:
# advanced_model을 적용하기 위한 agent.invoke()의 예시

# 최소 11개의 메시지(HumanMessage 또는 AIMessage)를 포함하는 history 준비
from langchain_core.messages import HumanMessage, AIMessage

# HumanMessage와 AIMessage를 번갈아 가며 11개 생성 (예시)
long_history = [
    HumanMessage(content=f"질문 {i+1}") if i % 2 == 0 else AIMessage(content=f"답변 {i+1}") 
    for i in range(11) # 메시지 개수: 11개 (11 > 10 조건 충족)
]

In [7]:
long_history

[HumanMessage(content='질문 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 6', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 7', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 8', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 10', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 11', additional_kwargs={}, response_metadata={})]

In [9]:
from langchain_core.messages import HumanMessage

# 이전 대화 기록 (11개의 메시지)
# long_history = [...] (이전 코드에서 이미 생성된 리스트)

# 1. 최종 사용자 질문을 HumanMessage로 만듭니다.
final_message = HumanMessage(content="이 긴 대화의 맥락에서 다음 질문에 답해줘.")

# 2. 전체 대화 기록을 하나의 'messages' 리스트로 합칩니다.
full_conversation = long_history + [final_message]

# 3. 에이전트에게 'messages' 키를 사용하여 전체 메시지 리스트를 전달합니다.
#    (LangChain Agents/Graph에서 대화 기록을 처리하는 가장 표준적인 방법)
agent.invoke(
    {
        "messages": full_conversation # 모든 메시지를 이 키에 통합하여 전달
    }
)

{'messages': [HumanMessage(content='질문 1', additional_kwargs={}, response_metadata={}, id='f498d1f7-1f23-410d-9d9c-35f5f20135be'),
  AIMessage(content='답변 2', additional_kwargs={}, response_metadata={}, id='57c3a6c9-9668-4fa7-b648-0c34f6d08cdf'),
  HumanMessage(content='질문 3', additional_kwargs={}, response_metadata={}, id='0346f4e1-0485-494a-a904-844140da8806'),
  AIMessage(content='답변 4', additional_kwargs={}, response_metadata={}, id='195e922f-dfb6-4997-b713-aed85e01282a'),
  HumanMessage(content='질문 5', additional_kwargs={}, response_metadata={}, id='3899bf99-2f7d-47f7-b5f1-ba3596d86c51'),
  AIMessage(content='답변 6', additional_kwargs={}, response_metadata={}, id='34168264-cad7-4e58-9910-dcc4cec48471'),
  HumanMessage(content='질문 7', additional_kwargs={}, response_metadata={}, id='3561d03e-1ebc-492d-a0a9-14fd3e15b04b'),
  AIMessage(content='답변 8', additional_kwargs={}, response_metadata={}, id='a72b3db4-7220-49cc-85c0-ab1f3d70747c'),
  HumanMessage(content='질문 9', additional_kwargs